In [ ]:
# ! pip install google
# ! pip install --user google.cloud
# ! pip install --user google.cloud.speech

In [1]:
import os
import io
import pickle
import time

import pandas as pd
import numpy as np

from pydub import AudioSegment
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types
from multiprocessing.dummy import Pool

In [2]:
with open("../data/Pickles/streets.pkl", "rb") as fp:
    streets = pickle.load(fp)

FileNotFoundError: [Errno 2] No such file or directory: '../data/Pickles/streets.pkl'

In [ ]:
pool = Pool(12) 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./api_key.json"   
client = speech.SpeechClient()

## With Street Name Context

In [ ]:
def get_desired_files(directory_name):
    desired_files = []
    for filename in os.listdir(directory_name):
        if (os.path.getsize(directory_name + '/' + filename) < 2_600_000) 
                                                & (filename.endswith('.wav')):
            desired_files.append(directory_name + '/' + filename)
    return desired_files

In [ ]:
def transcribe_audio_context(file_name):
    transcript = ''
    conf = 0
    # Loads the audio into memory
    with io.open(file_name, 'rb') as audio_file:
        content = audio_file.read()
        audio = types.RecognitionAudio(content=content)

    #speech_to_text
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        language_code='en-US',
        model="video",
        speech_contexts = [{
                        "phrases": np.random.choice(streets, 5000)
                         }]
    )
 

In [ ]:
# Detects speech in the audio file
def transcribe_audio_context(config, audio):
    
    response = client.recognize(config, audio)

    for result in response.results:
        transcript = result.alternatives[0].transcript
        confidence = result.alternatives[0].confidence

    time.sleep(1)
    return transcript, confidence, file_name

In [ ]:
pool = Pool(12) 

list_of_transcripts_context = pool.map(transcribe_audio_context, 
                                       get_desired_files('../audio_data/audio_files/wav_files/'))
pool.close()
pool.join()

captions_context = [a[0] for a in list_of_transcripts_context if a[0] != '']
confidence_context = [a[1] for a in list_of_transcripts_context if a[0] != '']
names_context = [a[2] for a in list_of_transcripts_context if a[0] != '']

data_context = {'transcripts': captions_context, 
                'confidence': confidence_context}
df_context = pd.DataFrame(data_context)
df_context.to_csv('../data/Data/transcribed_radio_with_street_context.csv')
df_context.head(25)

In [ ]:
np.mean(df_context['confidence'])

## Without Street Name Context

In [ ]:
def audio_file_prep(file_name):
    transcript = ''
    conf = 0
    # Loads the audio into memory
    with io.open(file_name, 'rb') as audio_file:
        content = audio_file.read()
        audio = types.RecognitionAudio(content=content)

    #speech_to_text
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        language_code='en-US',
        model="video"
    )

    return config, audio

In [ ]:
 # Detects speech in the audio file
def transcribe_audio(config, audio):
    
    response = client.recognize(config, audio)

    for result in response.results:
        transcript = result.alternatives[0].transcript
        confidence = result.alternatives[0].confidence

    time.sleep(3)
    
    return transcript, confidence, file_name

In [ ]:
pool = Pool(12) 

list_of_transcripts = pool.map(transcribe_audio, 
                               audio_file_prep('../audio_data/audio_files/wav_files/'))
pool.close()
pool.join()

captions = [a[0] for a in list_of_transcripts if a[0] != '']
confidence = [a[1] for a in list_of_transcripts if a[0] != '']
names = [a[2] for a in list_of_transcripts if a[0] != '']

data = {'transcripts': captions, 
        'confidence': confidence}
df = pd.DataFrame(data)
df.to_csv('../data/Data/transcribed_radio.csv')
df.head(25)

In [ ]:
np.mean(df['confidence'])

>Code adopted from [NY General Assembly DSI radio-to-location repository](https://github.com/mchbmn/radio-to-location)
